# XGBoost

One of the models we chose to implement for the purpose of lung cancer classification was XGBoost, a gradient boosting algorithm.  
This is the fastest implementation of gradient boosting and dominates tabular datasets on classification predictive modeling problems.

We start by importing relevant libraries and dropping useless columns from our CSV.

In [4]:
from kfold_and_metrics import *
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

In [5]:
df = pd.read_csv("final.csv")
df = df.drop(columns=["id"])
df.head()

,patient_id,diagnostics_Image-original_Mean,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,diagnostics_Image-interpolated_Mean,diagnostics_Image-interpolated_Minimum,diagnostics_Image-interpolated_Maximum,diagnostics_Mask-interpolated_VoxelNum,diagnostics_Mask-interpolated_VolumeNum,diagnostics_Mask-interpolated_Maximum,...,diagnostics_Mask-interpolated_BoundingBox_2,diagnostics_Mask-interpolated_BoundingBox_3,diagnostics_Mask-interpolated_BoundingBox_4,diagnostics_Mask-interpolated_BoundingBox_5,diagnostics_Mask-interpolated_CenterOfMassIndex_0,diagnostics_Mask-interpolated_CenterOfMassIndex_1,diagnostics_Mask-interpolated_CenterOfMassIndex_2,diagnostics_Mask-interpolated_CenterOfMass_0,diagnostics_Mask-interpolated_CenterOfMass_1,diagnostics_Mask-interpolated_CenterOfMass_2
0,LIDC-IDRI-0001,-826.943929,5905,1,-417.494203,-990.291016,1038.270874,909,2,237.087921,...,0.0,13.0,11.0,10.0,17.041265,16.108666,4.184319,128.652843,34.787644,-229.881362
1,LIDC-IDRI-0001,-826.943929,4613,1,-405.581777,-982.456726,949.768005,699,1,221.953705,...,0.0,13.0,11.0,10.0,17.041265,16.108666,4.184319,128.652843,34.787644,-229.881362
2,LIDC-IDRI-0001,-826.943929,4955,1,-410.236759,-990.291016,1038.270874,772,1,237.087921,...,0.0,13.0,11.0,10.0,17.041265,16.108666,4.184319,128.652843,34.787644,-229.881362
3,LIDC-IDRI-0001,-826.943929,5498,1,-416.576321,-990.291016,1038.270874,841,2,237.087921,...,0.0,13.0,11.0,10.0,17.041265,16.108666,4.184319,128.652843,34.787644,-229.881362
4,LIDC-IDRI-0002,-826.943929,10351,1,-546.359139,-1007.657349,1020.174988,749,1,160.687653,...,0.0,13.0,11.0,10.0,17.041265,16.108666,4.184319,128.652843,34.787644,-229.881362


### Parameter Hypertuning

In order to prevent overfitting, we hypertuned our model on the values of the number of estimators, max depth and max samples per leaf.

In [6]:
best_auc = 0
best = {}

for n_est in range(25, 201, 25):
    for m_depth in range(5, 56, 10):
        for m_child_weight in range(5, 26, 5):
            params = {'n_estimators': n_est, 'max_depth': m_depth, 'min_child_weight': m_child_weight}
            print("Current parameter combination:")
            for parameter, value in params.items():
                print(f"\t{parameter}: {value}")
            print()
            
            model = XGBClassifier(**params)
            auc_results = k_fold_cv(model, df, metric_funcs=[roc_auc_score], pca_components=100, k_fold_verbose=True)
            auc_average, auc_std = weighted_avg_and_std(np.array(auc_results['roc_auc_score']))
            if auc_average > best_auc:
                best_auc = auc_average
                best = params
                print("New best parameter combination:")
                print(f"AUC = {best_auc}")
                for parameter, value in best.items():
                    print(f"\t{parameter}: {value}")
                print()

Current parameter combination:
	n_estimators: 25
	max_depth: 5
	min_child_weight: 5

Performing K-Fold CV: 1 2 3 4 5 6 7 8 9 10
New best parameter combination:
AUC = 0.5737885806654792
	n_estimators: 25
	max_depth: 5
	min_child_weight: 5

Current parameter combination:
	n_estimators: 25
	max_depth: 5
	min_child_weight: 10

Performing K-Fold CV: 1 2 3 4 5 6 7 8 9 10
New best parameter combination:
AUC = 0.5816399160465863
	n_estimators: 25
	max_depth: 5
	min_child_weight: 10

Current parameter combination:
	n_estimators: 25
	max_depth: 5
	min_child_weight: 15

Performing K-Fold CV: 1 2 3 4 5 6 7 8 9 10
New best parameter combination:
AUC = 0.5979175368932592
	n_estimators: 25
	max_depth: 5
	min_child_weight: 15

Current parameter combination:
	n_estimators: 25
	max_depth: 5
	min_child_weight: 20

Performing K-Fold CV: 1 2 3 4 5 6 7 8 9 10
Current parameter combination:
	n_estimators: 25
	max_depth: 5
	min_child_weight: 25

Performing K-Fold CV: 1 2 3 4 5 6 7 8 9 10
Current parameter com

After running, we got the following values:

In [7]:
print("Results of the grid search parameter hypertunning:")
for parameter, value in best.items():
    print(f"\t{parameter}: {value}")

Results of the grid search parameter hypertunning:
	n_estimators: 150
	max_depth: 45
	min_child_weight: 5


In [17]:
best = {
    'n_estimators': 150,
	'max_depth': 45,
	'min_child_weight': 5,
}
model = XGBClassifier(**best)
scores = k_fold_cv(model, df, pca_components=100)
metrics_results = mean_std_results_k_fold_CV(scores)
metrics_results

,metric,mean,std
0,f1_score,0.438945,0.066279
1,accuracy_score,0.645724,0.034034
2,roc_auc_score,0.589324,0.031055
